In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [2]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['rush_hour','member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [3]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/Casual_RushMetro/landmarks.csv'
bs = set_data(file)

In [4]:
bs.columns

Index(['time_diff', 'temperature', 'humidity', 'windspeed', 'miles',
       'metro_dist', 'landmark_dist_start', 'landmark_dist_end', 'temp', 'hum',
       ...
       'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'weather_cat_1', 'weather_cat_2',
       'weather_cat_3'],
      dtype='object', length=303)

In [5]:
bs=bs.rename(columns = {'member_type_Registered':'member_type'})
bs = bs.drop(['temperature','windspeed','humidity'], 1)
bs.head()

,time_diff,miles,metro_dist,landmark_dist_start,landmark_dist_end,temp,hum,wind,rush_hour_1,member_type,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
1110598,4.850,0.619956,0.125465,1.329181,0.916964,4.141651,45.6667,13.458625,0,1,...,0,0,1,0,0,0,0,1,0,0
585262,15.517,1.882613,0.007178,2.592814,0.916964,25.370000,75.7500,13.209050,0,1,...,0,0,0,0,1,0,0,0,1,0
1077200,11.983,1.144273,0.450642,0.987390,0.485473,13.345849,55.4583,12.791171,0,1,...,0,0,0,0,1,0,0,1,0,0
977815,3.633,0.277219,0.328093,0.306432,0.485473,9.533021,71.6087,23.391710,0,1,...,0,0,1,0,0,0,0,0,1,0
551683,12.667,0.585465,0.586599,1.082599,0.839915,23.803349,81.7500,14.916411,0,1,...,1,0,0,0,0,0,0,0,1,0


In [6]:
X = bs.drop('member_type', 1)
y = bs['member_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((914885, 299), (304962, 299), (914885,), (304962,))

### GridSearch Logistic Regression

In [7]:
n_estimators = 125
class_weight = [{0:.675, 1:.325}, {0:.7, 1:.3}, {0:.68, 1:.32}]
features = ['sqrt','log2',20,30,40]
min_leaf_samples = [50,75,100,125,150]
param_grid = {'class_weight':class_weight, 'max_features': features, 'min_samples_leaf':min_leaf_samples}

rf1 = RandomForestClassifier(n_estimators, n_jobs=-1)
rf_gs1 = RandomizedSearchCV(rf1, param_grid, n_iter=15, cv=3, scoring='roc_auc')
rf_gs1.fit(X_train, y_train)

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=125, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=15, n_jobs=1,
          param_distributions={'class_weight': [{0: 0.675, 1: 0.325}, {0: 0.7, 1: 0.3}, {0: 0.68, 1: 0.32}], 'max_features': ['sqrt', 'log2', 20, 30, 40], 'min_samples_leaf': [50, 75, 100, 125, 150]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='roc_auc', verbose=0)

In [8]:
pred1 = rf_gs1.predict(X_test)
score1 = rf_gs1.score(X_test, y_test)
train_score1 = rf_gs1.score(X_train, y_train)

In [9]:
score1, train_score1

(0.86111110693796611, 0.8875662318422719)

In [10]:
confusion_matrix(y_test, pred1)

array([[ 27967,  22973],
       [ 14462, 239560]])

In [11]:
print(classification_report(y_test, pred1))

             precision    recall  f1-score   support

          0       0.66      0.55      0.60     50940
          1       0.91      0.94      0.93    254022

avg / total       0.87      0.88      0.87    304962



In [12]:
rf_gs1.best_params_

{'class_weight': {0: 0.68, 1: 0.32},
 'max_features': 40,
 'min_samples_leaf': 50}

In [13]:
roc_auc_score(y_test, pred1)

0.74604318816639981